## **Use Case Overview**

Imagine you’re part of an R&D team that needs to merge structured data (e.g., experimental results, market trends) from Microsoft Fabric with unstructured documents (e.g., research reports, engineering notes) in SharePoint, then validate these findings against external references (e.g., Bing) and a high-quality “ground truth” internal knowledge stores (e.g., Azure AI Search).

This was a classic Retrieval-Augmented Generation (RAG) scenario—multiple data sources must be queried in real time and cross-checked for consistency. However, by leveraging Azure AI agent services (an agetic enterprise-ready microservices approach) alongside frameworks like Semantic Kernel, we can evolve beyond basic RAG into a mostly autonomous, agentic system. In this design, Agentic RAG and the Reflection Pattern enable each agent to iteratively refine its output until it’s confident in delivering a high-quality, validated answer—paving the way for intelligent automation that continually learns and improves.

To summarize, you’re not only bringing data to the AI but also bringing AI to the data, thus maximizing the value of your knowledge stores. By leveraging state-of-the-art retrieval solutions like Azure AI Search, while also tapping sources such as SharePoint (unstructured data) and Fabric (structured data), you can harness your most valuable asset—data—to achieve new levels of insight and automation. 

**In this demo, we have two Azure AI Agents (extending beyond a single-agent architecture):**

+ DataRetrievalAgent: Has access to Microsoft Fabric (for structured data) and SharePoint (for unstructured documents). Its job is to gather relevant internal data: for example, “failure rates of Material X in high-temperature tests,” or “engineering notes on prior tests.”

- ValidationInsightsAgent Has access to Bing / Azure Cognitive Search for external references and can run a “reflection” or “validation” step. Its job is to cross-check what was returned by the first agent and highlight missing or conflicting information. ValidationInsightsAgent has access to highly curated knowledge sources (e.g., Azure AI Search) for validating the accuracy or truthfulness of the information it receives from DataRetrievalAgent.


**Moving from a single-agent setup to a multi-agent system is now simpler than ever with Semantic Kernel. The general flow looks like this:**

1. The user asks a question (e.g., “Retrieve historical failure rates for Material X in extreme temperatures and cross-check if new standards or conflicting data exist.”).
2. The DataRetrievalAgent fetches structured data from Fabric (e.g., lab test results, analytics) and unstructured docs from SharePoint (e.g., research memos, engineering notes).
3. The ValidationInsightsAgent then queries Bing/Azure Search to verify or supplement the results. Employ a reflection pattern, where it iterates over the combined results, looking for gaps or inconsistencies. If needed, it loops back to the DataRetrievalAgent for clarifications or additional data.

Finally, the user receives a validated, summarized answer that merges internal data with external cross-checks. Thanks to the agents’ back-and-forth reflection.

### **Why This Matters**

+ **Reduced Manual Research**: Instead of manually sifting through multiple data silos and external search engines, the AI Agents automate data gathering and vetting.
+ **Higher Confidence**: Validation ensures data accuracy and highlights missing pieces, improving R&D decision-making.
+ **Enterprise-Grade Security**: Each agent can enforce On-Behalf-Of (OBO) authentication to protect sensitive data (e.g., only pulling data the user is authorized to see).
In the Jupyter Notebook

When you run the code in this Jupyter notebook:

(TODO)

In [1]:
import os
import re
import time
import logging
import json
from datetime import datetime as pydatetime
from typing import Any, List, Dict, Optional
from dotenv import load_dotenv
import asyncio
from datetime import timedelta

# Azure AI Projects
from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError

# semantic kernel
from semantic_kernel.contents import AuthorRole
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.open_ai.run_polling_options import RunPollingOptions

# Load environment variables from .env file
load_dotenv()

# configure logging
from utils.ml_logging import get_logger

logger = get_logger()

# set the directory to the location of the script
try:
    target_directory = os.getenv("TARGET_DIRECTORY", os.getcwd())  # Use environment variable if available
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        logging.error(f"Directory does not exist: {target_directory}")
except Exception as e:
    logging.exception(f"An error occurred while changing directory: {e}")

### **Create Client and Load Azure AI Foundry**

Here, we initialize the Azure AI client using DefaultAzureCredential. This allows us to authenticate and connect to the Azure AI service.


In [2]:
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential())

### **1. Creating Azure AI Agents: FabricDataRetrievalAgent**

The DataRetrievalAgent is responsible for internal data retrieval, combining structured data from Microsoft Fabric with unstructured documents from SharePoint. This agent ensures that research teams can efficiently access critical R&D insights, such as historical failure rates, experimental results, and engineering notes—all while maintaining secure and authorized access controls.

Agent Capabilities
+ ✅ Structured Data Retrieval → Queries Microsoft Fabric for experiment logs, test results, and structured analytics.
+ ✅ Unstructured Document Search → Fetches relevant reports, blueprints, and research notes from SharePoint.
+ ✅ OBO Authentication → Uses On-Behalf-Of (OBO) authentication to ensure users can only access data they are permitted to view.

For a detailed breakdown of how to create a single Azure AI Agent and configure its data (tools) connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).

In [8]:
from azure.core.exceptions import ServiceRequestError
from azure.ai.projects.aio import AIProjectClient

async def get_connection_id(client: AIProjectClient, env_var: str) -> Optional[str]:
    """
    Retrieves the connection object using a connection name stored in an environment variable.

    Args:
        client: The Azure AI Project client.
        env_var (str): The environment variable holding the connection name.

    Returns:
        Connection object if found, otherwise raises an error.
    """
    connection_name = os.getenv(env_var)
    if not connection_name:
        logger.error(f"Missing environment variable: '{env_var}'")
        raise ValueError(f"Environment variable '{env_var}' is required.")

    try:
        connection = await client.connections.get(connection_name=connection_name)
        logger.info(f"Retrieved Connection ID for {env_var}: {connection.id}")
        return connection
    except Exception as e:
        logger.error(f"Failed to retrieve connection for {env_var}: {e}")
        raise

In [9]:
from azure.ai.projects.models import (
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # # Retrieve and add SharePoint Tool
    # sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    # toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # Retrieve and add Fabric Tool
    fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    toolset.add(FabricTool(connection_id=fabric_connection.id))

    logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-23 22:17:07,228 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_FABRIC: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/glucose_data_fabric (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_FABRIC: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/glucose_data_fabric
2025-03-23 22:17:07,236 - micro - MainProcess - INFO     Successfully created ToolSet with SharePoint and Fabric tools. (3024975452.py:<module>:21)
INFO:micro:Successfully created ToolSet with SharePoint and Fabric tools.


In [ ]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="FabricDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and integrating structured data from Microsoft Fabric. "
        "This includes performance metrics, experiment results, and analytics data. "
        "The agent is designed to assist in research and development by providing accurate, relevant, and actionable data. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for alternative queries or data sources."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving structured data exclusively from Microsoft Fabric. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research and development efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Structured Data (Microsoft Fabric):** \n"
        "   - Retrieve data from Microsoft Fabric when the query involves numerical metrics, performance statistics, or structured analytics.\n"
        "   - Data is available to support comparisons of the accuracy and reliability of two glucose monitoring products (Product A and Product B). "
        "This structured data evaluates performance across different glucose ranges and includes MARD percentages, accuracy within ±20 mg/dL/±20%, and the number of readings for each product.\n"
        "   - Example: clinical glucose monitoring studies between Product A or Product B.\n\n"
        
        "2. **Integrated Queries:** \n"
        "   - If the query requires multiple structured metrics, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "3. **Fallback Behavior:** \n"
        "   - If no relevant data is found in Microsoft Fabric, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for alternative queries or data sources (if applicable).\n"
        "     - Example: 'No relevant data was found for the requested query in Microsoft Fabric. Consider refining your query or exploring other data sources.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data from Microsoft Fabric.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Structured Data Retrieval for R&D",
        "data_source": "Microsoft Fabric (structured metrics only)",
    },
)

# Print the agent's run ID (agent ID)
print(f"DataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

DataRetrievalAgent Run ID: asst_2cjXDXvsx61XtikQFdpsHUNk


## Sharepoint Agent 

In [11]:
from azure.ai.projects.models import (
    SharepointTool,
    FabricTool,
    ToolSet,
)

# Initialize Azure AI Agent settings
dataretrievalagent_settings = AzureAIAgentSettings.create()

toolset = ToolSet()

try:
    # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # # Retrieve and add Fabric Tool
    # fabric_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_FABRIC")
    # toolset.add(FabricTool(connection_id=fabric_connection.id))

    logger.info("Successfully created ToolSet with SharePoint and Fabric tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-23 22:27:05,422 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint
2025-03-23 22:27:05,429 - micro - MainProcess - INFO     Successfully created ToolSet with SharePoint and Fabric tools. (2239274105.py:<module>:21)
INFO:micro:Successfully created ToolSet with SharePoint and Fabric tools.


In [13]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="SharePointDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and analyzing unstructured documents from SharePoint. "
        "This includes research papers, legal documents, and product engineering files (PDFs). "
        "The agent is designed to assist in research and development by providing accurate, relevant, and actionable insights. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for alternative queries or data sources."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and analyzing unstructured documents exclusively from SharePoint. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research and development, legal analysis, and product engineering efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Unstructured Data (SharePoint):** \n"
        "   - Retrieve documents from SharePoint when the query involves research papers, legal documents, or product engineering files (PDFs).\n"
        "   - Focus on extracting key insights, summaries, and actionable information from the retrieved documents.\n"
        "   - Example: 'Retrieve research papers on Material X used in high-temperature environments,' or 'Find legal documents related to patent filings for Product A.'\n\n"
        
        "2. **Document Types:** \n"
        "   - Research Papers: Summarize findings, methodologies, and conclusions.\n"
        "   - Legal Documents: Extract key clauses, compliance requirements, and patent-related information.\n"
        "   - Product Engineering Files: Highlight design notes, test results, and engineering decisions.\n\n"
        
        "3. **Integrated Queries:** \n"
        "   - If the query spans multiple document types, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "4. **Fallback Behavior:** \n"
        "   - If no relevant data is found in SharePoint, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for alternative queries or data sources (if applicable).\n"
        "     - Example: 'No relevant data was found for the requested query in SharePoint. Consider refining your query or exploring other data sources.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable documents from SharePoint.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "5. **Document Context:** Ensure that extracted insights are presented with sufficient context to maintain their relevance and accuracy.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Unstructured Data Retrieval for R&D, Legal, and Engineering",
        "data_source": "SharePoint",
    },
)

# Print the agent's run ID (agent ID)
print(f"SharePointDataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

SharePointDataRetrievalAgent Run ID: asst_kTtpnCZGYWammSC1PyYO6ljp


## Web Retriever

In [14]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

    # # Dynamically construct the PDF file path using os.path.join
    # pdf_file_path = os.path.join(target_directory, "data", "product_data", "ProductATechncialArchitecture.pdf")
    # logger.info(f"Using PDF file path: {pdf_file_path}")

    # file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    # vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
    #     file_ids=[file.id], name="my_vectorstore"
    # )

    # # 2. Create file search tool with uploaded resources
    # file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    # toolset.add(file_search)
    # logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-23 22:31:34,238 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding
2025-03-23 22:31:34,244 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (261821761.py:<module>:19)
INFO:micro:Bing Grounding Tool added successfully.
2025-03-23 22:31:34,250 - micro - MainProcess - INFO     Successfully created ToolSet with Bing and File Search tools. (261821761.py:<module>:46)
INFO:micro:Successfully created ToolSet with Bing and File Search tools.


In [15]:
dataretrievalagent_settings_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="BingDataRetrievalAgent",
    description=(
        "An AI agent specialized in retrieving and analyzing real-time data from the web using Bing. "
        "This includes retrieving up-to-date information, news, research articles, and other publicly available web content. "
        "The agent is designed to assist in research and decision-making by providing accurate, relevant, and actionable insights. "
        "If no relevant data is found, the agent must clearly indicate this and provide suggestions for refining the query."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a research-focused AI assistant responsible for retrieving and analyzing real-time data exclusively from the web using Bing. "
        "Your goal is to provide precise, well-referenced, and relevant responses to support research, decision-making, and analysis efforts.\n\n"
        
        "### Data Retrieval & Prioritization\n"
        "1. **Real-Time Web Data (Bing):** \n"
        "   - Use Bing to retrieve real-time data, including news, research articles, and other publicly available web content.\n"
        "   - Focus on extracting key insights, summaries, and actionable information from the retrieved web pages.\n"
        "   - Example: 'Retrieve the latest news on advancements in AI technology,' or 'Find research articles on the impact of climate change on agriculture.'\n\n"
        
        "2. **Source Evaluation:** \n"
        "   - Prioritize credible and authoritative sources (e.g., academic journals, government websites, reputable news outlets).\n"
        "   - Avoid using unreliable or unverified sources.\n\n"
        
        "3. **Integrated Queries:** \n"
        "   - If the query requires multiple pieces of information, retrieve and integrate the results for a comprehensive response.\n"
        "   - Ensure clarity in presenting combined insights.\n\n"
        
        "4. **Fallback Behavior:** \n"
        "   - If no relevant data is found on the web, respond with:\n"
        "     - A clear statement that no relevant data was found.\n"
        "     - Suggestions for refining the query or exploring alternative approaches.\n"
        "     - Example: 'No relevant data was found for the requested query on the web. Consider refining your query or specifying additional details.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Always prioritize retrieving the most current and applicable data from credible sources.\n"
        "2. **Clarity & Transparency:** Clearly indicate the data source(s) used and any limitations in the available information.\n"
        "3. **Fallback Handling:** If no relevant data is found, provide a professional and helpful fallback response as outlined above.\n"
        "4. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "5. **Source Context:** Ensure that extracted insights are presented with sufficient context to maintain their relevance and accuracy.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Real-Time Web Data Retrieval for Research and Decision-Making",
        "data_source": "Bing (web search)",
    },
)

# Print the agent's run ID (agent ID)
print(f"BingDataRetrievalAgent Run ID: {dataretrievalagent_settings_definition.id}")

BingDataRetrievalAgent Run ID: asst_E7bYR4yLZXBdQdodvd5prSYc


## Verifier Agent

In [18]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
verifieragent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # # Retrieve and add the Bing Grounding Tool
    # bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    # toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    # logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

        # Retrieve and add SharePoint Tool
    sharepoint_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SHAREPOINT")
    toolset.add(SharepointTool(connection_id=sharepoint_connection.id))

    # Dynamically construct the PDF file path using os.path.join
    pdf_file_path = os.path.join(target_directory, "data", "product_data", "ProductATechncialArchitecture.pdf")
    logger.info(f"Using PDF file path: {pdf_file_path}")

    file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )

    # 2. Create file search tool with uploaded resources
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    toolset.add(file_search)
    logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-23 22:47:04,122 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_SHAREPOINT: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/ContosoAgentDemoSharepoint
2025-03-23 22:47:04,127 - micro - MainProcess - INFO     Using PDF file path: c:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf (1457801407.py:<module>:36)
INFO:micro:Using PDF file path: c:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf
2025-03-23 22:47:09,960 - mic

In [19]:
verifieragent_settings_definition = await project_client.agents.create_agent(
    model=verifieragent_settings.model_deployment_name,
    name="VerifierAgent",
    description=(
        "An AI agent specialized in reviewing and validating data retrieved from multiple sources. "
        "The agent ensures that the data is consistent, relevant, and aligned with the user's query. "
        "It is capable of identifying discrepancies, highlighting missing information, and providing a final decision on the validity of the data. "
        "If the data is valid and complete, the agent will finalize the response with 'Approved'."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are a validation-focused AI assistant responsible for reviewing and validating data retrieved from multiple sources. "
        "Your goal is to ensure the data is consistent, relevant, and aligned with the user's query. "
        "You must identify discrepancies, highlight missing information, and provide a final decision on the validity of the data.\n\n"
        
        "### Validation Process\n"
        "1. **Review Data from Multiple Sources:** \n"
        "   - Analyze data retrieved by other agents (e.g., SharePoint, Bing, Fabric) to ensure consistency and relevance.\n"
        "   - Cross-check the data against the user's query to confirm alignment.\n"
        "   - Example: 'Verify if the retrieved research papers and legal documents align with the user's request for compliance standards.'\n\n"
        
        "2. **Identify Discrepancies:** \n"
        "   - Highlight any conflicting information or gaps in the data.\n"
        "   - Provide actionable feedback to guide further data retrieval if necessary.\n\n"
        
        "3. **Final Decision:** \n"
        "   - If the data is valid and complete, respond with 'Approved'.\n"
        "   - If the data is incomplete or inconsistent, provide a clear explanation and suggest next steps.\n"
        "   - Example: 'The data retrieved from SharePoint and Bing is consistent and aligns with the user's query. Approved.'\n"
        "     or 'The data retrieved from Fabric is incomplete. Missing insights on Material X. Please refine the query.'\n\n"
        
        "### Response Quality\n"
        "1. **Accuracy & Relevance:** Ensure the validation process is thorough and the final decision is based on accurate and relevant data.\n"
        "2. **Clarity & Transparency:** Clearly indicate the sources reviewed and any limitations in the data.\n"
        "3. **Professionalism:** Present findings in a structured and concise manner to facilitate decision-making.\n"
        "4. **Actionable Feedback:** Provide clear next steps if the data is incomplete or inconsistent.\n"
    ),
    toolset=toolset,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.5,
    top_p=1,
    metadata={
        "use_case": "Data Validation and Approval for Multi-Agent Systems",
        "data_source": "Multiple sources",
    },
)

# Print the agent's run ID (agent ID)
print(f"VerifierAgent Run ID: {verifieragent_settings_definition.id}")

VerifierAgent Run ID: asst_nkhC85ADcuFVvhLqC76mCXc0


In [7]:
dataretrievalagent = AzureAIAgent(
    client=project_client,
    definition=dataretrievalagent_settings_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

thread = await project_client.agents.create_thread()

USER_INPUTS = [
    "How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?"
    "What are the latest trends in R&D?",
    "What are the key features from Dexcom G7 CGM System?",
   ]
try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await dataretrievalagent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in dataretrievalagent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logging.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logging.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** How does Product A compare to Product B in terms of MARD percentage across different glucose ranges?What are the latest trends in R&D?

🤖 **Agent:** ### Comparison of Product A and Product B in Terms of MARD Percentage

In terms of Mean Absolute Relative Difference (MARD) across different glucose ranges, Product A and Product B exhibit distinct performance metrics. Unfortunately, I don't have the specific data from the Microsoft Fabric system at this moment, but typically, MARD is used to evaluate the accuracy of glucose monitoring devices. Lower MARD percentages indicate higher accuracy.

### Latest Trends in R&D

Recent trends in R&D for diabetes technology include advancements in automated insulin delivery systems. For instance, the Omnipod 5, an automated insulin delivery system, was approved by the FDA in January 2022. This system integrates a tubeless insulin pump with a continuous glucose monitoring sensor and uses a control algorithm to adjust insulin delivery【5:1†s

### **2. Creating Azure AI Agents: ValidationInsightsAgent**

The ValidationInsightsAgent is designed to validate and cross-check the data retrieved by the DataRetrievalAgent. It accesses external references such as Bing and Azure Cognitive Search to verify and supplement the internal data. Additionally, it leverages highly curated knowledge sources (e.g., Azure AI Search) to ensure the accuracy and truthfulness of the information, using a reflection or validation step to highlight missing or conflicting details.

Agent Capabilities:

+ ✅ External Reference Verification → Queries Bing and Azure Cognitive Search for real-time validation.
+ ✅ Reflection & Validation Step → Iteratively reviews and refines the information received from the DataRetrievalAgent.
+ ✅ Curated Knowledge Validation → Uses Azure AI Search to confirm the accuracy and reliability of internal data.

For a detailed breakdown of how to create a ValidationInsightsAgent and configure its external tools and connections, please refer to:
📌 [01-single-agents-with-azure-ai-agents.ipynb](01-single-agents-with-azure-ai-agents.ipynb).


In [10]:
from azure.ai.projects.models import (
    BingGroundingTool,
    AzureAISearchTool,
    FileSearchTool,
    VectorStore,
    OpenAIFile
)

# Initialize Azure AI Agent settings
validationinsightagent_settings = AzureAIAgentSettings.create()

# Create a ToolSet to manage tools
toolset = ToolSet()

try:
    # Retrieve and add the Bing Grounding Tool
    bing_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_BING")
    toolset.add(BingGroundingTool(connection_id=bing_connection.id))
    logger.info("Bing Grounding Tool added successfully.")

    # Retrieve and add the Azure AI Search Tool
    # search_connection = await get_connection_id(project_client, "TOOL_CONNECTION_NAME_SEARCH")
    # azure_ai_search_connection = AzureAISearchTool(
    #     index_connection_id=search_connection.id,
    #     index_name="ai-agentic-index"
    # )
    # toolset.add(azure_ai_search_connection)
    # logger.info("Azure AI Search Tool added successfully.")

    # Dynamically construct the PDF file path using os.path.join
    pdf_file_path = os.path.join(target_directory, "data", "product_data", "ProductATechncialArchitecture.pdf")
    logger.info(f"Using PDF file path: {pdf_file_path}")

    file: OpenAIFile = await project_client.agents.upload_file_and_poll(file_path=pdf_file_path, purpose="assistants")
    vector_store: VectorStore = await project_client.agents.create_vector_store_and_poll(
        file_ids=[file.id], name="my_vectorstore"
    )

    # 2. Create file search tool with uploaded resources
    file_search = FileSearchTool(vector_store_ids=[vector_store.id])

    toolset.add(file_search)
    logger.info("Azure AI Search Tool added successfully.")
 

    logger.info("Successfully created ToolSet with Bing and File Search tools.")
except Exception as e:
    logger.error(f"Failed to create ToolSet: {e}")
    raise

2025-03-20 15:40:22,801 - micro - MainProcess - INFO     Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding (2375541312.py:get_connection_id:22)
INFO:micro:Retrieved Connection ID for TOOL_CONNECTION_NAME_BING: /subscriptions/47f1c914-e299-4953-a99d-3e34644cfe1c/resourceGroups/rg-mukeshag-5297_ai/providers/Microsoft.MachineLearningServices/workspaces/zhuoqunli-5026/connections/agentsbinggrounding
2025-03-20 15:40:22,808 - micro - MainProcess - INFO     Bing Grounding Tool added successfully. (1615353572.py:<module>:19)
INFO:micro:Bing Grounding Tool added successfully.
2025-03-20 15:40:22,813 - micro - MainProcess - INFO     Using PDF file path: c:\Users\pablosal\Desktop\azure-ai-agent-services-demo\data\product_data\ProductATechncialArchitecture.pdf (1615353572.py:<module>:32)
INFO:micro:Using P

In [11]:
# Create or update a new Validation Insights Agent
validationinsightsagent_definition = await project_client.agents.create_agent(
    model=dataretrievalagent_settings.model_deployment_name,
    name="ValidationInsightsAgent",
    description=(
        "An AI agent designed to validate and refine R&D insights by cross-checking "
        "both internal enterprise data sources (e.g., file search vector store, SharePoint, Fabric) "
        "and external public data (Bing, Azure AI Search). It uses a reflection pattern "
        "to ensure response accuracy, consistency, and proper citations."
    ),
    instructions=(
        "### Role & Objective\n"
        "You are the 'Validation Insights' AI assistant, responsible for validating insights by "
        "retrieving and cross-checking data from internal enterprise sources and external web searches. "
        "Your responses must be accurate, well-referenced, and actionable."
        "\n\n"
        "### Data Retrieval & Prioritization\n"
        "1. **Primary Data Source**: Always consult internal enterprise sources first (File Search Vector Store). "
        "2. **External Validation**: If internal data is insufficient or requires verification, utilize Bing and Azure AI Search."
        "\n\n"
        "### Reflection & Validation\n"
        "1. **Evaluate Consistency**: After retrieving data, compare sources to identify inconsistencies.\n"
        "2. **Perform Refinement**: If contradictions or gaps exist, conduct a second retrieval pass to ensure accuracy."
        "\n\n"
        "### Response Quality\n"
        "1. **Accuracy & Transparency**: Always prioritize factual correctness and completeness in responses.\n"
        "2. **Citations & References**: Clearly cite the sources (internal or external) used in your response.\n"
        "3. **Clarity & Professionalism**: Deliver insights concisely and in a structured format. Be transparent if data validation is pending."
    ),
    toolset=toolset,
    # Prefer the internal file search vector store by default
    tool_resources=file_search.resources,
    headers={"x-ms-enable-preview": "true"},
    temperature=0.7,
    top_p=1,
    metadata={
        "use_case": "Cross-Validation and Insight Generation for R&D",
        "data_source": "Internal (Fabric, SharePoint) and External (Bing, Azure AI Search)",
        "response_validation": (
            "Employ a reflection step to cross-check data accuracy "
            "and provide clear citations in final responses."
        )
    },
)

print(f"ValidationInsightsAgent Run ID: {validationinsightsagent_definition.id}")


ValidationInsightsAgent Run ID: asst_cflIDuNZR0fOpTJp59zhhDdn


In [12]:
validation_agent = AzureAIAgent(
    client=project_client,
    definition=validationinsightsagent_definition,
    polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
)

# Create a conversation thread for the agent
thread = await project_client.agents.create_thread()

# Define a list of user inputs designed to trigger both internal search (e.g., product architecture) 
# and external market trend validation.
USER_INPUTS = [
    "What are the characteristics and architecture of Product A?",
    "What are the key features from Dexcom G7 CGM System?",
]

try:
    for user_input in USER_INPUTS:
        # Add the user input as a chat message
        await validation_agent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"👤 **User:** {user_input}\n")
        
        # Invoke the agent for the specified thread and stream the response
        async for content in validation_agent.invoke(thread_id=thread.id):
            # Only print non-tool messages
            if content.role != AuthorRole.TOOL:
                print(f"🤖 **Agent:** {content.content}\n")
                
except HttpResponseError as e:
    try:
        error_json = json.loads(e.response.content)
        logger.error(f"❌ **Error Message:** {error_json.get('Message')}")
    except json.JSONDecodeError:
        logger.error(f"❌ **Non-JSON Error Content:** {e.response.content}")

👤 **User:** What are the characteristics and architecture of Product A?

🤖 **Agent:** ### Characteristics and Architecture of Product A

**Characteristics:**

1. **Functionality**: Product A is an integrated continuous glucose monitoring (iCGM) system designed to replace traditional blood glucose monitoring by providing real-time interstitial glucose readings【4:0†source】.

2. **Accuracy**: Demonstrates a high level of accuracy with a Mean Absolute Relative Difference (MARD) of approximately 8.5% to 9.2% during varying glucose conditions【4:1†source】.

3. **Usability**: User-friendly with a mobile application that provides real-time data, customizable alerts, and compatibility with other diabetes management devices【4:4†source】.

4. **Durability**: Engineered for long-term sensor wear (up to 10 days) with a hypoallergenic adhesive patch【4:4†source】.

5. **Adaptability**: Includes a future-ready adaptive machine learning algorithm for personalized insulin dosing and lifestyle adjustments【4

### **2. Creating Multi Agent System**

The following sample demonstrates how to create an OpenAI assistant using either Azure OpenAI or OpenAI, a chat completion agent and have them participate in a group chat to work towards the user's requirement.


In [10]:
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

In [ ]:
import os
import asyncio
import json
import logging
from typing import List, Dict

from azure.identity.aio import DefaultAzureCredential
from azure.core.exceptions import HttpResponseError
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt

# Custom Termination Strategy
class ApprovalTerminationStrategy(TerminationStrategy):
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        last_msg = history[-1]
        return (
            last_msg.name in [a.name for a in self.agents] and 
            ("approved" in last_msg.content.lower())
        )

# Kernel Creation
def create_kernel() -> Kernel:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    kernel.add_service(
        AzureChatCompletion(
            deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_ID"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        )
    )
    return kernel

RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.

    Choose only from these participants:
    - {RETRIEVER_NAME}
    - {EVALUATOR_NAME}

    Rules:
    - {RETRIEVER_NAME} retrieves relevant data.
    - {EVALUATOR_NAME} evaluates data.
    - Workflow terminates if Evaluator says 'approved'.

    Respond with ONLY participant's name.

    History:
    {{{{$history}}}}
    """
)

# Utility Logging
def log_agent_invocation(agent_name, role, input_data, output_data, metadata):
    trace_data = {
        "agent_name": agent_name,
        "role": role,
        "input": input_data,
        "output": output_data,
        "metadata": metadata,
    }
    logger.info("AGENT INVOCATION:\n" + json.dumps(trace_data, indent=4))

# Main Async Function
async def main():
    async with DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds) as client:

        ValidationInsightsAgentID = "asst_cflIDuNZR0fOpTJp59zhhDdn"
        DataRetrievalAgentID = "asst_BN7AvpIntZuA4s7nI2wQB4ri"

        dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
        validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

        agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
        agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

        chat = AgentGroupChat(
            agents=[agent_retriever, agent_evaluator],
            termination_strategy=ApprovalTerminationStrategy(
                maximum_iterations=4,
                agents=[agent_evaluator],
            ),
            selection_strategy=KernelFunctionSelectionStrategy(
                function=selection_function,
                kernel=create_kernel(),
                result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
                agent_variable_name="agents",
                history_variable_name="history",
            ),
        )

        agent_name_to_id = {
            RETRIEVER_NAME: ValidationInsightsAgentID,
            EVALUATOR_NAME: DataRetrievalAgentID,
        }

        system_message = "Your system message here..."
        await chat.add_chat_message(message=system_message)

        user_message = "What are the characteristics and architecture of Product A?"
        await chat.add_chat_message(message=user_message)

        last_input = user_message

        try:
            async for content in chat.invoke():
                agent_name = content.name or "Unknown"
                agent_role = content.role.name
                agent_output = content.content

                # Extracting citations clearly from AnnotationContent items
                citations = []
                for item in content.items:
                    if item.content_type == 'annotation':
                        citation = {
                            "quote": item.quote,
                            "url": item.url
                        }
                        citations.append(citation)

                # Construct structured metadata
                metadata = {
                    "citations": citations,
                    "agent_id": agent_name_to_id.get(agent_name, 'unknown'),
                }

                # Log invocation with structured metadata
                log_agent_invocation(agent_name, agent_role, last_input, agent_output, metadata)

                # Print structured information clearly
                print(f"AGENT ROLE: {agent_role}")
                print(f"AGENT NAME: {agent_name}")
                print("OUTPUT:", agent_output)
                print("CITATIONS:")
                for citation in citations:
                    print(f"  - Quote: {citation['quote']}, URL: {citation['url']}")
                print("-" * 80)

                last_input = agent_output

        except HttpResponseError as err:
            logger.error(f"Http Error during conversation: {err}")

        except Exception as e:
            logger.error(f"General Error during conversation: {e}")

        finally:
            # Safely reset after ensuring no active agent
            if chat._is_active:
                chat._is_active = False  # force inactive state if stuck
            await chat.reset()


In [12]:
await main() 

2025-03-23 16:51:54,321 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "What are the characteristics and architecture of Product A?",
    "output": "Product A is an advanced integrated continuous glucose monitoring (iCGM) system. Here are the key characteristics and architecture details:\n\n### Characteristics\n\n1. **Sensor Module**:\n   - **Design**: Factory-calibrated electrochemical sensor measuring interstitial glucose every 5 minutes.\n   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40\u2013400 mg/dL.\n   - **Protective Membrane**: Micro-porous membrane minimizes biofouling and interference.\n   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear\u30105:0\u2020source\u3011.\n\n2. **Transmitter and Communication**:\n   - **Transmitter**: Low-power, battery-operated, converting analog signals to digital data every 5 minutes.\n   - **Wireless 

AGENT ROLE: ASSISTANT
AGENT NAME: ValidationInsightsAgent
OUTPUT: Product A is an advanced integrated continuous glucose monitoring (iCGM) system. Here are the key characteristics and architecture details:

### Characteristics

1. **Sensor Module**:
   - **Design**: Factory-calibrated electrochemical sensor measuring interstitial glucose every 5 minutes.
   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40–400 mg/dL.
   - **Protective Membrane**: Micro-porous membrane minimizes biofouling and interference.
   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear【5:0†source】.

2. **Transmitter and Communication**:
   - **Transmitter**: Low-power, battery-operated, converting analog signals to digital data every 5 minutes.
   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.
   - **Data Buffering**: Stores up to 24 hours of data during connectivity losses【5:4†source】.

3. **Software and Analytics**:
   - **Mobile App**

2025-03-23 16:52:13,694 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "DataRetrievalAgent",
    "role": "ASSISTANT",
    "input": "Product A is an advanced integrated continuous glucose monitoring (iCGM) system. Here are the key characteristics and architecture details:\n\n### Characteristics\n\n1. **Sensor Module**:\n   - **Design**: Factory-calibrated electrochemical sensor measuring interstitial glucose every 5 minutes.\n   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40\u2013400 mg/dL.\n   - **Protective Membrane**: Micro-porous membrane minimizes biofouling and interference.\n   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear\u30105:0\u2020source\u3011.\n\n2. **Transmitter and Communication**:\n   - **Transmitter**: Low-power, battery-operated, converting analog signals to digital data every 5 minutes.\n   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.\n   - **Data Buffering**

AGENT ROLE: ASSISTANT
AGENT NAME: DataRetrievalAgent
OUTPUT: Product A is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following characteristics and architecture:

### Characteristics

1. **Sensor Module**:
   - **Design**: Factory-calibrated, electrochemical sensor that measures interstitial glucose every 5 minutes.
   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40–400 mg/dL.
   - **Protective Membrane**: Minimizes biofouling and interference.
   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear【5:1†source】.

2. **Transmitter and Communication**:
   - **Transmitter**: Low-power, battery-operated, converts analog signals to digital data every 5 minutes.
   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.
   - **Data Buffering**: Stores up to 24 hours of data during connectivity losses【5:1†source】.

3. **Software and Analytics**:
   - **Mobile App**: Offers real-time sensor r

2025-03-23 16:52:25,846 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "Product A is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following characteristics and architecture:\n\n### Characteristics\n\n1. **Sensor Module**:\n   - **Design**: Factory-calibrated, electrochemical sensor that measures interstitial glucose every 5 minutes.\n   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40\u2013400 mg/dL.\n   - **Protective Membrane**: Minimizes biofouling and interference.\n   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear\u30105:1\u2020source\u3011.\n\n2. **Transmitter and Communication**:\n   - **Transmitter**: Low-power, battery-operated, converts analog signals to digital data every 5 minutes.\n   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.\n   - **Data Buffering**: Stores up to 24

AGENT ROLE: ASSISTANT
AGENT NAME: ValidationInsightsAgent
OUTPUT: Product A is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following characteristics and architecture:

### Characteristics

1. **Sensor Module**:
   - **Design**: Factory-calibrated, electrochemical sensor that measures interstitial glucose every 5 minutes.
   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40–400 mg/dL.
   - **Protective Membrane**: Minimizes biofouling and interference.
   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear【5:0†source】.

2. **Transmitter and Communication**:
   - **Transmitter**: Low-power, battery-operated, converts analog signals to digital data every 5 minutes.
   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.
   - **Data Buffering**: Stores up to 24 hours of data during connectivity losses【5:4†source】.

3. **Software and Analytics**:
   - **Mobile App**: Offers real-time sen

2025-03-23 16:52:37,298 - micro - MainProcess - INFO     AGENT INVOCATION:
{
    "agent_name": "ValidationInsightsAgent",
    "role": "ASSISTANT",
    "input": "Product A is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following characteristics and architecture:\n\n### Characteristics\n\n1. **Sensor Module**:\n   - **Design**: Factory-calibrated, electrochemical sensor that measures interstitial glucose every 5 minutes.\n   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40\u2013400 mg/dL.\n   - **Protective Membrane**: Minimizes biofouling and interference.\n   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear\u30105:0\u2020source\u3011.\n\n2. **Transmitter and Communication**:\n   - **Transmitter**: Low-power, battery-operated, converts analog signals to digital data every 5 minutes.\n   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.\n   - **Data Buffering**: Stores up to 24

AGENT ROLE: ASSISTANT
AGENT NAME: ValidationInsightsAgent
OUTPUT: Product A is a sophisticated integrated continuous glucose monitoring (iCGM) system with the following characteristics and architecture:

### Characteristics

1. **Sensor Module**:
   - **Design**: Factory-calibrated, electrochemical sensor that measures interstitial glucose every 5 minutes.
   - **Core**: Enzyme-coated electrode for high sensitivity across a glucose range of 40–400 mg/dL.
   - **Protective Membrane**: Minimizes biofouling and interference.
   - **Adhesive and Patch**: Hypoallergenic for up to 10 days of wear【5:0†source】.

2. **Transmitter and Communication**:
   - **Transmitter**: Low-power, battery-operated, converts analog signals to digital data every 5 minutes.
   - **Wireless Connectivity**: Secure Bluetooth with AES-256 encryption.
   - **Data Buffering**: Stores up to 24 hours of data during connectivity losses【5:4†source】.

3. **Software and Analytics**:
   - **Mobile App**: Offers real-time sen

## Creting Streamlit App 

In [ ]:
# 2. Import Necessary Modules
# Import the required modules in your Python script:

import os
import asyncio
import streamlit as st
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel import Kernel
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents.azure_ai import AzureAIAgent
from semantic_kernel.agents.strategies import TerminationStrategy, KernelFunctionSelectionStrategy
from semantic_kernel.functions import KernelFunctionFromPrompt


In [ ]:
# 3. Define the Approval Termination Strategy
# Create a custom termination strategy to end the conversation based on specific criteria:

class ApprovalTerminationStrategy(TerminationStrategy):
    def __init__(self, agents, maximum_iterations=10):
        super().__init__(maximum_iterations=maximum_iterations)
        self.agents = agents

    async def should_agent_terminate(self, agent, history) -> bool:
        last_msg = history[-1]
        return (last_msg.name in [a.name for a in self.agents]) and ("approved" in last_msg.content.lower())


In [ ]:
# 4. Initialize the Kernel with Chat Completion
# Set up the Semantic Kernel with an Azure OpenAI chat completion service:

def _create_kernel_with_chat_completion() -> Kernel:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    kernel = Kernel()
    AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
    AZURE_OPENAI_API_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
    AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")
    AZURE_AOAI_CHAT_MODEL_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_MODEL_DEPLOYMENT", "gpt-4")

    kernel.add_service(
        service=AzureChatCompletion(
            deployment_name=AZURE_AOAI_CHAT_MODEL_DEPLOYMENT,
            api_key=AZURE_OPENAI_KEY,
            endpoint=AZURE_OPENAI_API_ENDPOINT,
            api_version=AZURE_OPENAI_API_VERSION,
        )
    )
    return kernel


In [ ]:
# 5. Define the Selection Function Prompt
# Create a prompt to determine the next participant in the conversation:

RETRIEVER_NAME = "ValidationInsightsAgent"
EVALUATOR_NAME = "DataRetrievalAgent"

selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
Determine which participant takes the next turn in a conversation based on the most recent participant.
State only the name of the participant to take the next turn.
No participant should take more than one turn in a row.

Choose only from these participants:
- {RETRIEVER_NAME}
- {EVALUATOR_NAME}

Always follow these rules when selecting the next participant:
- {RETRIEVER_NAME} retrieves the document or relevant data for the query.
- After {RETRIEVER_NAME}, it is {EVALUATOR_NAME}'s turn to evaluate the content.
- After {EVALUATOR_NAME}, the workflow may terminate if 'approved'.

History:
{{{{$history}}}}
""",
)


In [ ]:
# 6. Initialize Agents and Chat
# Set up the agents and the chat interface:

async def initialize_agents():
    creds = DefaultAzureCredential()
    client = await AzureAIAgent.create_client(credential=creds)

    ValidationInsightsAgentID = "asst_kdFT72VdYH0YpoG3tJ5lmoFy"
    DataRetrievalAgentID = "asst_Wo0GJ9MpmvkfRPNwllC7bYFS"

    dataretrieval_def = await client.agents.get_agent(agent_id=DataRetrievalAgentID)
    validation_def = await client.agents.get_agent(agent_id=ValidationInsightsAgentID)

    agent_retriever = AzureAIAgent(client=client, definition=dataretrieval_def)
    agent_evaluator = AzureAIAgent(client=client, definition=validation_def)

    chat = AgentGroupChat(
        agents=[agent_retriever, agent_evaluator],
        termination_strategy=ApprovalTerminationStrategy(
            maximum_iterations=10,
            agents=[agent_evaluator],
        ),
        selection_strategy=KernelFunctionSelectionStrategy(
            function=selection_function,
            kernel=_create_kernel_with_chat_completion(),
            result_parser=lambda result: str(result.value[0]) if result.value else EVALUATOR_NAME,
            agent_variable_name="agents",
            history_variable_name="history",
        ),
    )

    return chat


In [ ]:
# 7. Build the Streamlit Interface
# Create the Streamlit interface to interact with the agents:

async def main():
    st.title("Multi-Agent Chat Interface")

    if "chat" not in st.session_state:
        st.session_state.chat = await initialize_agents()
        st.session_state.history = []

    user_input = st.chat_input("Enter your message:")
    if user_input:
        st.session_state.history.append({"role": "user", "content": user_input})
        await st.session_state.chat.add_chat_message(message=user_input)

        async for content in st.session_state.chat.invoke():
            agent_name = content.name or "Unknown"
            agent_role = content.role.name
            agent_output = content.content

            st.session_state.history.append({"role": agent_role, "name": agent_name, "content": agent_output})

            st.chat_message(agent_role).write(f"**{agent_name}:** {agent_output}")

        if st.session_state.chat.is_complete:
            st.write("Conversation has been approved and terminated.")
            st.session_state.chat.reset()
            st.session_state.history = []

if __name__ == "__main__":
    asyncio.run(main())


8. Run the Streamlit Application

Save your script (e.g., app.py) and run it using Streamlit:

In [1]:
!streamlit run src/chatapp/app.py


^C
